# FmKorea 하이닉스 커뮤니티 크롤링 코드

### 실행순서

1. 1번째 셀 실행 (크롤링을 위한 함수 정의)
2. 2번째 셀 실행 (검색 키워드 "삼성전자"로 크롤링 실행, df_samsung 데이터프레임 return)
3. 3번째 셀 실행 (검색 키워드 "삼전"으로 크롤링 실행, df_samjeon 데이터프레임 return)
4. 4번째 셀 실행 (생성된 df 2개에서 중복되는 게시글 삭제 및 csv 파일생성)

In [2]:
import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import re

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/143.0.0.0 Safari/537.36"
    ),
    "Referer": "https://www.fmkorea.com/",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
}

# 날짜 문자열을 정규화(통일)하는 함수
# - "17:27" 처럼 시간만 있으면 오늘 날짜(YYYY-MM-DD)로 치환
# - "2026.01.12" 처럼 점(.)으로 된 날짜는 "-"로 변경
def normalize_date(date_str: str) -> str:
    today = datetime.now().strftime("%Y-%m-%d")
    s = (date_str or "").strip()
    if re.match(r"^\d{1,2}:\d{2}$", s):
        return today
    if "." in s:
        return s.replace(".", "-")
    return s

# 검색 결과 페이지를 순회하며 게시글 리스트를 수집해서 DataFrame으로 반환
# start_page ~ end_page: 수집할 페이지 범위
def crawl_one(session: requests.Session, url_base: str, source_name: str,
              start_page=1, end_page=10, sleep_sec=2) -> pd.DataFrame:
    data = {"탭": [], "제목": [], "글쓴이": [], "날짜": [], "조회": [], "추천": [], "댓글수": [],
            "post_url": [], "source": []}

    for page in range(start_page, end_page + 1):
        url = url_base.format(page)

        r = session.get(url, timeout=20)
        r.raise_for_status()

        soup = bs(r.text, "lxml")
        rows = soup.select("table.bd_lst.bd_tb_lst.bd_tb tbody tr")
        if not rows:
            print(f"[{source_name}] {page}페이지: rows=0 → 중단")
            break

        page_added = 0
        for tr in rows:
            cate_a = tr.select_one("td.cate a")
            title_a = tr.select_one("td.title a.hx")
            author_a = tr.select_one("td.author a")
            reply_a = tr.select_one("td.title a.replyNum")
            time_td = tr.select_one("td.time")
            mno_tds = tr.select("td.m_no")

            if not (cate_a and title_a and author_a and time_td and len(mno_tds) >= 2):
                continue
            
            reply_cnt = int(reply_a.get_text(strip=True)) if reply_a else 0 

            views = mno_tds[0].text.strip()
            votes = mno_tds[1].text.strip()

            href = title_a.get("href", "")
            post_url = "https://www.fmkorea.com" + href if href.startswith("/") else href

            data["탭"].append(cate_a.get_text(strip=True))
            data["제목"].append(title_a.get_text(" ", strip=True))
            data["글쓴이"].append(author_a.get_text(strip=True))
            data["날짜"].append(normalize_date(time_td.get_text(strip=True)))
            data["조회"].append(int(views.replace(",", "")) if views else 0)
            data["추천"].append(int(votes.replace(",", "")) if votes else 0)
            data["댓글수"].append(reply_cnt) 
            data["post_url"].append(post_url)
            data["source"].append(source_name)

            page_added += 1

        print(f"[{source_name}] {page}페이지 완료 / 이번 페이지 {page_added}개 / 누적 {len(data['post_url'])}개")

        if page_added == 0:
            print(f"[{source_name}] {page}페이지: page_added=0 → 중단")
            break

        time.sleep(sleep_sec)

    return pd.DataFrame(data)


## 크롤링 실행

https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&search_target=title_content&page=1

- 코드 실행시, 위 링크로 접속한뒤에 다시 링크를 복사한뒤 붙여넣어주세요.
- 붙여넣고 링크 맨 뒤, page 부분을 {}로 수정해주세요.
- 코드 실행시, 페이지 새로고침 후 실행해주세요.

In [16]:
# 검색키원드 "하이닉스"

url_base_hynix = "https://www.fmkorea.com/search.php?mid=stock&category=2997203870&search_keyword=%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&search_target=title&listStyle=list&page={}"
#450
# Session을 쓰면 동일한 세션(쿠키 등)을 유지한 채 여러 페이지를 연속 요청할 수 있음 
session_hynix = requests.Session()
session_hynix.headers.update(headers)  # 세션별 쿠키/헤더 유지

start_page = 401
end_page = 500
#447
# 크롤링 실행
df_hynix = crawl_one(
    session_hynix,
    url_base_hynix,
    "하이닉스",
    start_page=start_page,
    end_page=end_page
)

df_hynix.tail()

[하이닉스] 401페이지 완료 / 이번 페이지 20개 / 누적 20개
[하이닉스] 402페이지 완료 / 이번 페이지 20개 / 누적 40개
[하이닉스] 403페이지 완료 / 이번 페이지 20개 / 누적 60개
[하이닉스] 404페이지 완료 / 이번 페이지 20개 / 누적 80개
[하이닉스] 405페이지 완료 / 이번 페이지 20개 / 누적 100개
[하이닉스] 406페이지 완료 / 이번 페이지 20개 / 누적 120개
[하이닉스] 407페이지 완료 / 이번 페이지 20개 / 누적 140개
[하이닉스] 408페이지 완료 / 이번 페이지 20개 / 누적 160개
[하이닉스] 409페이지 완료 / 이번 페이지 20개 / 누적 180개
[하이닉스] 410페이지 완료 / 이번 페이지 20개 / 누적 200개
[하이닉스] 411페이지 완료 / 이번 페이지 20개 / 누적 220개
[하이닉스] 412페이지 완료 / 이번 페이지 20개 / 누적 240개
[하이닉스] 413페이지 완료 / 이번 페이지 20개 / 누적 260개
[하이닉스] 414페이지 완료 / 이번 페이지 20개 / 누적 280개
[하이닉스] 415페이지 완료 / 이번 페이지 20개 / 누적 300개
[하이닉스] 416페이지 완료 / 이번 페이지 20개 / 누적 320개
[하이닉스] 417페이지 완료 / 이번 페이지 20개 / 누적 340개
[하이닉스] 418페이지 완료 / 이번 페이지 20개 / 누적 360개
[하이닉스] 419페이지 완료 / 이번 페이지 20개 / 누적 380개
[하이닉스] 420페이지 완료 / 이번 페이지 20개 / 누적 400개
[하이닉스] 421페이지 완료 / 이번 페이지 20개 / 누적 420개
[하이닉스] 422페이지 완료 / 이번 페이지 20개 / 누적 440개
[하이닉스] 423페이지 완료 / 이번 페이지 20개 / 누적 460개
[하이닉스] 424페이지 완료 / 이번 페이지 20개 / 누적 480개
[하이닉스] 425페이지 완료 / 이번 페이지 20개 / 누적 500개
[하이닉

,탭,제목,글쓴이,날짜,조회,추천,댓글수,post_url,source
1995,국내주식,하이닉스 는 위로 깨네 마네하고있는데,이말내,2024-10-10,230,1,0,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
1996,국내주식,삼전과 하이닉스 차이 하.,인간쓰레기ㅜㅜ,2024-10-10,216,3,2,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
1997,국내주식,하이닉스 18.8층 매도벽은 삼전급이네,HM7,2024-10-10,269,1,2,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
1998,국내주식,sk 하이닉스 회심의 트라이중,켄.피셔,2024-10-10,235,1,0,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
1999,국내주식,SK 하이닉스 !!,Mokagold,2024-10-10,588,3,1,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스


## 크롤링 실행

https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%EC%82%BC%EC%A0%84&search_target=title_content&page=1

- 코드 실행시, 위 링크로 접속한뒤에 다시 링크를 복사한뒤 붙여넣어주세요.
- 붙여넣고 링크 맨 뒤, page 부분을 {}로 수정해주세요.
- 코드 실행시, 페이지 새로고침 후 실행해주세요.

In [17]:
# 검색키워드 "하닉"

url_base_hanic = "https://www.fmkorea.com/search.php?mid=stock&category=2997203870&search_keyword=%ED%95%98%EB%8B%89&search_target=title_content&listStyle=list&page={}"

# Session을 쓰면 동일한 세션(쿠키 등)을 유지한 채 여러 페이지를 연속 요청할 수 있음 
session_hanic = requests.Session()
session_hanic.headers.update(headers)  # 세션별 쿠키/헤더 유지

start_page = 401
end_page = 500

# 크롤링 실행
df_hanic = crawl_one(
    session_hanic,
    url_base_hanic,
    "하닉",
    start_page=start_page,
    end_page=end_page
)

df_hanic.tail()


[하닉] 401페이지 완료 / 이번 페이지 20개 / 누적 20개
[하닉] 402페이지 완료 / 이번 페이지 20개 / 누적 40개
[하닉] 403페이지 완료 / 이번 페이지 20개 / 누적 60개
[하닉] 404페이지 완료 / 이번 페이지 20개 / 누적 80개
[하닉] 405페이지 완료 / 이번 페이지 20개 / 누적 100개
[하닉] 406페이지 완료 / 이번 페이지 20개 / 누적 120개
[하닉] 407페이지 완료 / 이번 페이지 20개 / 누적 140개
[하닉] 408페이지 완료 / 이번 페이지 20개 / 누적 160개
[하닉] 409페이지 완료 / 이번 페이지 20개 / 누적 180개
[하닉] 410페이지 완료 / 이번 페이지 20개 / 누적 200개
[하닉] 411페이지 완료 / 이번 페이지 20개 / 누적 220개
[하닉] 412페이지 완료 / 이번 페이지 20개 / 누적 240개
[하닉] 413페이지 완료 / 이번 페이지 20개 / 누적 260개
[하닉] 414페이지 완료 / 이번 페이지 20개 / 누적 280개
[하닉] 415페이지 완료 / 이번 페이지 20개 / 누적 300개
[하닉] 416페이지 완료 / 이번 페이지 20개 / 누적 320개
[하닉] 417페이지 완료 / 이번 페이지 20개 / 누적 340개
[하닉] 418페이지 완료 / 이번 페이지 20개 / 누적 360개
[하닉] 419페이지 완료 / 이번 페이지 20개 / 누적 380개
[하닉] 420페이지 완료 / 이번 페이지 20개 / 누적 400개
[하닉] 421페이지 완료 / 이번 페이지 20개 / 누적 420개
[하닉] 422페이지 완료 / 이번 페이지 20개 / 누적 440개
[하닉] 423페이지 완료 / 이번 페이지 20개 / 누적 460개
[하닉] 424페이지 완료 / 이번 페이지 20개 / 누적 480개
[하닉] 425페이지 완료 / 이번 페이지 20개 / 누적 500개
[하닉] 426페이지 완료 / 이번 페이지 20개 / 누적 520개
[하닉] 427페이지 완료 /

,탭,제목,글쓴이,날짜,조회,추천,댓글수,post_url,source
1995,국내주식,네이버 차트는 좋은듯?,소신발언할게요,2025-10-28,3157,11,8,https://www.fmkorea.com/index.php?mid=stock&ca...,하닉
1996,국내주식,하닉 1시보고 안봤는데 언제 52왔냐,하이닉스매수해라,2025-10-28,295,0,0,https://www.fmkorea.com/index.php?mid=stock&ca...,하닉
1997,국내주식,"[단독]SK 하닉 , 연내 TC본더 추가발주 없다…장비업계 '비상'",관세할당,2025-10-28,1993,12,6,https://www.fmkorea.com/index.php?mid=stock&ca...,하닉
1998,국내주식,ETF가 확실히 안정감 지리긴 하네,지금지쳤나요,2025-10-28,810,1,1,https://www.fmkorea.com/index.php?mid=stock&ca...,하닉
1999,국내주식,하닉 은 진짜 모르겠다,삼전10만가자,2025-10-28,997,9,5,https://www.fmkorea.com/index.php?mid=stock&ca...,하닉


csv 파일 생성

In [18]:
# 두 데이터프레임 합치기

df_all = pd.concat([df_hynix, df_hanic], ignore_index=True) 
print(f"합치기 전: {len(df_all):,}")

# 중복 제거: URL 기준
df_all = df_all.drop_duplicates(subset=["post_url"], keep="first").reset_index(drop=True) 
print(f"URL 중복 제거 후: {len(df_all):,}")

# 날짜 내림차순 정렬
df_all["날짜_dt"] = pd.to_datetime(df_all["날짜"], errors="coerce")
df_all = (
    df_all.sort_values(by="날짜_dt", ascending=False, na_position="last")
          .drop(columns=["날짜_dt"])
          .reset_index(drop=True)
)

# 1,2,3,4,5로 이름 바꾸면서 저장
df_all.to_csv("../data/5.csv", index=False, encoding="utf-8-sig")
df_all.tail()

합치기 전: 4,000
URL 중복 제거 후: 4,000


,탭,제목,글쓴이,날짜,조회,추천,댓글수,post_url,source
3995,국내주식,하이닉스 3분기 실적발표일 언제인가요?,박부라더스,2024-10-10,461,0,4,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
3996,국내주식,한강 수혜주는 sk 하이닉스 입니다,웸반야마도는블락,2024-10-10,168,2,0,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
3997,국내주식,HBM 전쟁 2라운드도 SK 하이닉스 ‘선공,sadsdl,2024-10-10,789,5,0,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
3998,국내주식,SK 하이닉스 는,주링이,2024-10-10,2076,5,1,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스
3999,국내주식,"SK 하이닉스 , 창립 41주년...""HBM으로 세계 첫 1위""",sadsdl,2024-10-10,368,6,3,https://www.fmkorea.com/index.php?mid=stock&ca...,하이닉스


100페이지씩 나누어진 데이터를 하나의 csv로 결합

In [19]:
import pandas as pd
import glob

# 1) 합칠 CSV 파일들 (현재 폴더의 csv 전부면 이렇게)
files = [
    "../data/1.csv",
    "../data/2.csv",
    "../data/3.csv",
    "../data/4.csv",
    "../data/5.csv",
]

# 2) 모두 읽어서 합치기
dfs = [pd.read_csv(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

print("합치기 전:", len(df_all))

# 3) 중복 제거 (post_url 기준이 가장 확실)
df_all = df_all.drop_duplicates(subset=["post_url"], keep="first").reset_index(drop=True)  # [web:60]
print("중복 제거 후:", len(df_all))

# 4) 날짜 내림차순 정렬(안전하게 datetime 변환 후 정렬)
df_all["날짜_dt"] = pd.to_datetime(df_all["날짜"], errors="coerce")
df_all = (
    df_all.sort_values(by="날짜_dt", ascending=False, na_position="last")  # [web:102]
          .drop(columns=["날짜_dt"])
          .reset_index(drop=True)
)

# 5) 최종 저장
df_all.to_csv("../data/fm_hynix_normal.csv", index=False, encoding="utf-8-sig")

합치기 전: 19500
중복 제거 후: 19500
